In [103]:
# Import packages
import json
import asyncio

import pandas as pd
from openai import OpenAI

from google.cloud import bigquery
from google.cloud.exceptions import BadRequest


In [ ]:
# Set parameters
# BigQuery setup
project_id = 'PROJECT_ID'
client = bigquery.Client(project=project_id)
# Max GB to run a query
max_gb=30

# Openai setup
api_key = 'API_KEY'
base_url = 'https://api.openai.com/v1'
default_headers = {'Authorization': f'Bearer {api_key}'}
model_name = 'gpt-5-2025-08-07'

In [68]:
## Pre-prompting of agent
# System prompt
system_prompt = '''Your role is to translate requests to SQL queries for BigQuery.

IMPORTANT: You have access to tools to help you.

IMPORTANT BEHAVIOR RULES:
Always explain with one short sentence what you're going to do before using any tools
Don't just call tools without explanation.'''

# User rules
user_rules = 'Use USING for joins - when possible. Put SQL keywords in uppercase.'

In [74]:
## Define tools


# Declare python function
def execute_query(query):
    print('Query:', query)
    
    # Step 1: Dry run to validate query and check cost
    try:
        print('⚙️ Performing dry run...')
        job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
        dry_run_job = client.query(query, job_config=job_config)
        
        # Calculate cost in GB
        bytes_processed = dry_run_job.total_bytes_processed
        gb_processed = bytes_processed / (1024 ** 3)  # Convert bytes to GB
        print(f'💰 Query cost: {gb_processed:.2f} GB')
        
        # Check if query is too expensive
        if gb_processed > max_gb:
            return {
                'success': False,
                'error': f'Query cost is {gb_processed:.2f} GB which is too costly (max: {max_gb} GB)',
                'data': None
            }
        
        print('✅ Dry run successful, proceeding with execution...')
        
    except Exception as e:
        print('❌ Query failed, check the query syntax')
        return {
            'success': False,
            'error': f'Query validation failed: {str(e)}',
            'data': None,
        }
    
    # Step 2: Execute the actual query
    try:
        print('🚀 Executing query...')
        job = client.query(query)
        results = job.result()
        
        # Convert results to list of dictionaries for easier handling
        data = []
        for row in results:
            data.append(dict(row))
        
        print(f'✅ Query executed successfully! Returned {len(data)} rows')
        # Print the preview in a dataframe
        df = pd.DataFrame(data)
        display(df)
        
        return {
            'success': True,
            'error': None,
            'data': data
        }
        
    except Exception as e:
        return {
            'success': False,
            'error': f'Query execution failed: {str(e)}',
            'data': None
        }


# Describe function for OpenAI
openai_tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_query",
            "description": "Execute a SQL query on BigQuery. Returns the query results.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "BigQuery SQL query to estimate the cost of."
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

# Simple tool description - convert tool dict to string
tools_description = f"Available tools:\n{json.dumps(openai_tools, indent=2)}"

# Tool registry to link tool names to functions
TOOL_REGISTRY = {
    "execute_query": execute_query,
}


In [107]:
# Create agent
def run_agent(user_prompt, context='', privacy_mode=True):
    # Contact all prompts
    messages = [
        {"role": "system", "content": system_prompt + "\n" + tools_description},
        {"role": "user", "content": user_rules},
        {"role": "user", "content": user_prompt + "\n" + context}
    ]

    # Get response
    llm_client = OpenAI(api_key=api_key, base_url=base_url, default_headers=default_headers)
    running = True

    while running:
        response = llm_client.chat.completions.create(
            messages=messages,
            model=model_name,
            tools=openai_tools,
            parallel_tool_calls=False,
            stream=False,
        )

        # Get message
        message = response.choices[0].message
        print("🤖 Assistant:", message.content)

        # Get tool calls
        tool_calls = []
        if message.tool_calls:
            # List tool to call
            for tool_call in message.tool_calls:
                tool_calls.append({
                    "name": tool_call.function.name,
                    "args": tool_call.function.arguments,
                    "id": tool_call.id
                })
            
            # Add tool calls to messages
            messages.append({
                "role": "assistant",
                "content": message.content,
                "tool_calls": [
                    {
                        "id": tc.id,
                        "type": "function",
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments
                        }
                    } for tc in message.tool_calls
                ]
            })
        
        else:
            # Add assistant message to messages
            messages.append({
                "role": "assistant",
                "content": message.content
            })
        
        # Execute tool calls
        for tool_call in tool_calls:
            try:
                print(f"🛠️ Calling: {tool_call['name']}")
                result = TOOL_REGISTRY[tool_call["name"]](**json.loads(tool_call["args"]))

                if privacy_mode:
                    result = 'Results hidden due to privacy mode'

                # Send tool results back to LLM
                messages.append({
                    "role": "tool",
                    "content": json.dumps(result),
                    "tool_call_id": tool_call["id"]
                })
            except Exception as e:
                print(f"❌ Error calling tool {tool_call['name']}: {e}")
                messages.append({
                    "role": "tool",
                    "content": f"Error: {str(e)}",
                    "tool_call_id": tool_call["id"]
                })

        # If tools were called, keep running
        running = len(tool_calls) > 0
        
    # print("🤖 Assistant (final response): ", final_message.content)



In [118]:
# Launch a request to your agent
# Define if privacy mode is on or off (privacy mode = query results are not shared with the LLM)
privacy_mode = True

# Provide context
context = '''
Table: nao-dbt-demo.nao_corp.customers
Columns: customer_id, created_at
Table: nao-dbt-demo.nao_corp.retailers
Columns: retailer_id, created_at, cancelled_at
'''

# Enter a user prompt
user_prompt = '''
How many clients do I have? 
How many retailers do I have?
Answer the question in two distinct queries. and Execute one after the other.
''' 


In [119]:
# Run agent
run_agent(user_prompt, context, privacy_mode=False)


🤖 Assistant: I will execute a query to count the total number of clients in the customers table.
🛠️ Calling: execute_query
Query: SELECT COUNT(*) AS client_count
FROM `nao-dbt-demo.nao_corp.customers`;
⚙️ Performing dry run...
💰 Query cost: 0.00 GB
✅ Dry run successful, proceeding with execution...
🚀 Executing query...
✅ Query executed successfully! Returned 1 rows


,client_count
0,100


🤖 Assistant: I will execute a query to count the total number of retailers in the retailers table.
🛠️ Calling: execute_query
Query: SELECT COUNT(*) AS retailer_count
FROM `nao-dbt-demo.nao_corp.retailers`;
⚙️ Performing dry run...
💰 Query cost: 0.00 GB
✅ Dry run successful, proceeding with execution...
🚀 Executing query...
✅ Query executed successfully! Returned 1 rows


,retailer_count
0,60606


🤖 Assistant: Here are two separate queries executed sequentially:

1) Count of clients
SELECT COUNT(*) AS CLIENT_COUNT
FROM `nao-dbt-demo.nao_corp.customers`;

Result: 100

2) Count of retailers
SELECT COUNT(*) AS RETAILER_COUNT
FROM `nao-dbt-demo.nao_corp.retailers`;

Result: 60606
